# Comparación dinámicas SEIR  vs SEIRHVD

**Created by:** Samuel Ropert  
**Creation date:** 30/04/2020  
**Institution:** Computational Biology Lab - Fundación Ciencia y Vida, Chile  

El modelo SEIRHVD presenta comportamientos extraños para un modelo compartimental de este tipo. El objetivo de este notebook es compararlo con un modelo SEIR con los mismos parámetros, y de encontrar incosistencias será la herramienta para identificar el problema.


## Resumen:
SEIRHVD tenía un bug en la implementación de las ecuaciones diferenciales. Habían variables cambiadas, en vez de los infectados críticos y severos estaban ingresados los expuestos diarios y acumulados.

## Import Libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIR2 import SEIR
from class_SEIRHUVD6 import SEIRHVD 
from Quarantine import Quarantine

Linux


## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

## Simulation Parameters

In [2]:
beta = 0.2 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 500
# Population
population = 1000000
# Initial Active Infected 
I0 = 100
I_ac0 = 100
I_d0 = 0
# Kinetic Saturation: 0 for mass action mixing
k=0

### Quarantines

In [3]:
def alpha(t):
    return 1

# SEIRHVD params

In [4]:
#state parameters: 
pE_Ias = 1 # Transition from exposed to Asymptomatic Infected
tE_Ias = 3.0
tIas_R = 12.0

## Create simulation Object

In [5]:
simseir = SEIR(tsim=tsim,alpha=alpha,beta=beta,mu=mu,sigma=1/tE_Ias,gamma=1/tIas_R,I0=I0,I_ac0=I_ac0,I_d0=I_d0,population=population,expinfection=0)

In [6]:
simseirhvd = SEIRHVD(tsim,beta,mu=mu,alpha=alpha,I0=I0,I_ac0=I_ac0,I_d0=I_d0,population=population)
simseirhvd.tE_Ias = tE_Ias
simseirhvd.tIas_R = tIas_R

# Set infected transition probability 
simseirhvd.pE_Ias = 1 # Transition from exposed to Asymptomatic Infected
simseirhvd.pE_Imi = 0 # Transition from exposed to  Mild Infected
simseirhvd.pE_Icr = 0 # Transition from exposed to  Critical Infected
simseirhvd.pE_Ise = 0 # Transition from exposed to  Serious Infected
simseirhvd.setnewparams()

Compartimental model State parameters changed


## Simulate
The different scenarios are simulated in parallel threads 

In [7]:
simseir.integr_sci(0,tsim,0.01)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 334
     njev: 9
      nlu: 9
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 6.32455532e-07, 1.26491106e-06, 4.05010253e-03,
       8.09894016e-03, 1.21477778e-02, 5.26361540e-02, 9.31245302e-02,
       1.33612906e-01, 1.74101283e-01, 4.06157719e-01, 6.38214156e-01,
       8.70270592e-01, 1.10232703e+00, 1.33438346e+00, 1.77086250e+00,
       2.20734153e+00, 2.64382056e+00, 3.08029959e+00, 3.51677862e+00,
       4.04870727e+00, 4.58063593e+00, 5.11256458e+00, 5.64449324e+00,
       6.04970075e+00, 6.45490827e+00, 6.86011578e+00, 7.73367531e+00,
       8.60723485e+00, 9.48079438e+00, 1.04927290e+01, 1.15046636e+01,
       1.25165983e+01, 1.32210507e+01, 1.39255032e+01, 1.46299557e+01,
       1.78474554e+01, 1.86518303e+01, 1.94562053e+01, 2.02605802e+01,
       2.10649551e+01, 2.17075663e+01, 2.23501775e+01, 2.29927887e+01,
       2.53382333e+01, 2.76836780e+01, 3

In [8]:
simseirhvd.integr_sci(0,tsim,0.01)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 1018
     njev: 19
      nlu: 19
      sol: None
   status: 0
  success: True
        t: array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146,

# Simulation Analysis

## Plots
This libraries have predefined plot functions which plot the main epidemiological variables.
Each function has the following optional arguments:
* days [int] Number of days to display
* showparams [bool] Display simulation parameters 
* ylim [int] Limit the vertical axis
* norm [int/float] Normalize the results


### SEIR Plot with Active infected

In [9]:
plt.plot(simseir.t,simseir.S,label='S SEIR',color = 'blue',linestyle='dashed')
plt.plot(simseir.t,simseir.E,label='E SEIR',color = 'cyan',linestyle='dashed')
plt.plot(simseir.t,simseir.I,label='I SEIR',color = 'red',linestyle='dashed')
plt.plot(simseir.t,simseir.R,label='R SEIR',color = 'green',linestyle='dashed')

plt.plot(simseirhvd.t,simseirhvd.S,label='S SEIRHVD',color = 'blue')
plt.plot(simseirhvd.t,simseirhvd.E,label='E SEIRHVD',color = 'cyan')
plt.plot(simseirhvd.t,simseirhvd.I,label='I SEIRHVD',color = 'red')
plt.plot(simseirhvd.t,simseirhvd.R,label='R SEIRHVD',color = 'green')
plt.title('Epidemiological Plot')
plt.legend(loc=0)
plt.show()

# wtf SEIRHVD

## IC

In [ ]:
print(simseirhvd.S[0])
print(simseirhvd.E[0])
print(simseirhvd.I[0])
print(simseirhvd.R[0])
print(simseirhvd.H[0])
print(simseirhvd.V[0])
print(simseirhvd.D[0])
print(simseirhvd.B[0])

In [ ]:
simseirhvd.S[0]+simseirhvd.E[0]+simseirhvd.I[0]

In [ ]:
plt.plot(simseirhvd.S+simseirhvd.E+simseirhvd.Ias+simseirhvd.R)
plt.show()

In [ ]:
type(simseirhvd.alpha)

In [ ]:
plt.plot(range(100),simseirhvd.alpha(range(100)))

In [ ]:
print(simseirhvd.N)
print(simseir.N)

In [ ]:
simseirhvd.mu

In [ ]:
plt.plot(simseirhvd.t,simseirhvd.S,label='S SEIRHVD',color = 'blue')
plt.plot(simseirhvd.t,simseirhvd.E,label='E SEIRHVD',color = 'cyan')
plt.plot(simseirhvd.t,simseirhvd.Ias,label='Ias SEIRHVD',color = 'red')
plt.plot(simseirhvd.t,simseirhvd.Imi,label='Imi SEIRHVD')
plt.plot(simseirhvd.t,simseirhvd.Ise,label='Ise SEIRHVD')
plt.plot(simseirhvd.t,simseirhvd.Icr,label='Icr SEIRHVD')

plt.plot(simseirhvd.t,simseirhvd.R,label='R SEIRHVD',color = 'green')
plt.plot(simseirhvd.t,simseirhvd.H,label='H SEIRHVD',color = 'orange')
plt.plot(simseirhvd.t,simseirhvd.V,label='V SEIRHVD',color = 'orange')
plt.plot(simseirhvd.t,simseirhvd.B,label='B SEIRHVD',color = 'orange')
plt.plot(simseirhvd.t,simseirhvd.D,label='D SEIRHVD',color = 'orange')
plt.title('Epidemiological Plot')
plt.legend(loc=0)
plt.show()

In [ ]:
plt.plot(simseirhvd.t,simseirhvd.I,label='I SEIRHVD')
plt.plot(simseirhvd.t,simseirhvd.Ias,label='Ias SEIRHVD')
plt.plot(simseirhvd.t,simseirhvd.Imi,label='Imi SEIRHVD')
plt.plot(simseirhvd.t,simseirhvd.Ise,label='Ise SEIRHVD')
plt.plot(simseirhvd.t,simseirhvd.Icr,label='Icr SEIRHVD')
plt.title('Infected Plot')
plt.legend(loc=0)
plt.show()

Plot Active Infected

In [ ]:
for i in range(len(quarantines)):
    plt.plot(simulation[i].t,simulation[i].I,label='Quarantine: '+str(i))
plt.title('Quarantines')
plt.legend(loc=0)
plt.show()

### Plot Accumulated Infected

In [ ]:
for i in range(len(quarantines)):
    plt.plot(simulation[i].t,simulation[i].I_ac,label='Quarantine: '+str(i))

plt.title('Accumulated Infected')
plt.legend(loc=0)
plt.show()

### Plot New Daily Infected

In [ ]:
for i in range(len(quarantines)):
    plt.plot(simulation[i].t,simulation[i].I_d,label='Quarantine: '+str(i))

plt.title('New Daily Infected')
plt.legend(loc=0)
plt.show()

### Variables:
The simulation object contains several methods and variables with its results

In [ ]:
# Susceptibles
simulation[0].S
# Exposed
simulation[0].E
# Infected
simulation[0].I
# Recovered
simulation[0].R

## Peak Calculation

In [ ]:
# Peak Values per each scenario
simulation[0].peak

In [ ]:
# Peak day counted from the simulation beginning
simulation[0].peak_t